In [24]:
# * Use at your own risk *
# Configure the parameters and create info.text with the formatting specified below
# The code is unreliable sometimes (selenium doesnt click things, so re-running it is sometimes needed)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from time import sleep
import requests
import re

# Params
wait = {"implicit": 5, "forStreamFetch": 5, "forDownloadLink": 25, "betweenDownloads": 10 * 60}
monthsBack = 5 # Specify how many months in the past do you want the recording to capture (mm-monthsBack/yr to mm/yr)
with open("info.txt", encoding="utf-8") as file:
    courseName, studentID, password = file.read().split("\n") # info.txt should have the following format: [studentID]\n[password]\n[courseName]
    # courseName is the name of the course as it appears in my courses page
    

In [21]:
# Initialize driver and get page
chromedriver_autoinstaller.install()
options = Options()
options.add_argument("--use-fake-ui-for-media-stream")
driver = webdriver.Chrome(chrome_options=options)
driver.get("https://lms.ksu.edu.sa/")
driver.maximize_window()
driver.maximize_window()

# Options
driver.implicitly_wait(wait["implicit"])

In [22]:
# Login

driver.find_element_by_id("user_id").send_keys(studentID)
driver.find_element_by_id("password").send_keys(password)
driver.find_element_by_id("entry-login").click()

In [25]:
# Enter Course Page

course215 = driver.find_element_by_link_text(courseName)
link = course215.get_property("href")
driver.execute_script(f'window.open("{link}","_blank");')
driver.switch_to.window(driver.window_handles[1])

In [26]:
# Clicks recordings
driver.switch_to.frame(driver.find_element_by_id("collabUltraLtiFrame"))
driver.find_element_by_id("side-menu-toggle").click()
driver.find_element_by_xpath('//*[@id="side-menu"]/div/nav/ul/li[3]/a/span').click()


In [27]:
# Clicks recordings in range
driver.find_element_by_xpath('//*[@id="filter-toggle-145"]/span[1]/span').click()
driver.find_element_by_xpath('//*[@id="select-range-filter-145"]/span').click()


In [28]:
# Navigate to calendar
driver.find_element_by_xpath('//*[@id="startDate-145"]').click()

In [29]:
# Click month back monthsBack times
for _ in range(monthsBack):
    driver.find_element_by_xpath('/html/body/div[4]/div[1]/table/thead/tr[2]/th[1]').click()
# Click random day in month
driver.find_element_by_xpath('/html/body/div[4]/div[1]/table/tbody/tr[2]/td[1]').click()

In [30]:
# Helper functions
def returnToLinks():
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame(driver.find_element_by_id("collabUltraLtiFrame"))

def writeLog():
    global recordingsLog
    with open("recordings.log", "w") as log:
        log.write("\n".join(recordingsLog))
    
recordingsLog = []
def downloadVids():
    recordings = '//*[@id="body-content"]/div[2]/table/tbody/tr[{}]' # .format(2) is the first recording in the currently shown table, .format(1) is column names
    global recordingsLog
    videoXpath = '//*[@id="playback-video-playback-video_html5_api"]'
    i = 2
    cur = driver.find_elements_by_xpath(recordings.format(i)) # Empty list on finish
    while cur:
        cur = cur[0]

        # Compute name
        recordingsLog.append(cur.text)

        # CODE MIGHT BE WRONG I DIDNT TEST IT 
        name = re.sub('[^\w\-_\. ]', '_', recordingsLog[-1]) + ".mp4"

        # Click recording button then click popup
        element = cur.find_element_by_tag_name("td")
        element.find_element_by_tag_name("button").click()
        sleep(wait["forStreamFetch"])
        element.find_elements_by_tag_name("button")[1].click()

        # Switch window
        driver.switch_to.window(driver.window_handles[2])

        # Get url and download it
        sleep(wait["forDownloadLink"])
        url = driver.find_element_by_xpath('//*[@id="playback-video-playback-video_html5_api"]')
        url = url.get_attribute("src")
        r = requests.get(url, allow_redirects=True)
        with open(name, 'wb') as file:
            file.write(r.content)

        # Close and switch window and frame back
        returnToLinks()

        i += 1
        cur = driver.find_elements_by_xpath(recordings.format(i))
        
        writeLog()
        sleep(wait["betweenDownloads"])

In [ ]:
# MANUALLY NAVIGATE TO PAGE YOU WANT TO DOWNLOAD THEN RUN THIS CELL
downloadVids()

In [34]:
# Do not run this cell unless needed
returnToLinks()

In [144]:
# rightArrow = '//*[@id="main-content"]/footer/ol/li[4]/button/bb-svg-icon'
# pages = driver.find_element_by_xpath('//*[@id="main-content"]/footer/ol')
# pages.find_elements_by_tag_name("button")[1].click()
# # //*[@id="main-content"]/footer/ol/li[2]
# # //*[@id="main-content"]/footer/ol/li[3]
# # //*[@id="main-content"]/footer/ol/li[4]
# # Something somethign click next page

In [ ]:
# TODO: right arrow after finishing